In [2]:
import pandas as pd

employee_survey = pd.read_csv('data/raw/employee_survey_data.csv')
employee_survey.head(1)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0


In [3]:
manager_survey = pd.read_csv('data/raw/manager_survey_data.csv')
manager_survey.head(1)

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3


In [4]:
general_data = pd.read_csv('data/raw/general_data.csv')
general_data.head(1)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0


In [5]:
import zipfile
import os

def get_in_out_time(DATA_DIR: str = 'data/raw') -> tuple[pd.DataFrame, pd.DataFrame]:
    join = os.path.join
    with zipfile.ZipFile(join(DATA_DIR,'in_out_time.zip'), 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    in_time = pd.read_csv(join(DATA_DIR,'in_time.csv'))
    out_time = pd.read_csv(join(DATA_DIR, 'out_time.csv'))

    # rename first column to 'EmployeeID'
    in_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
    out_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

    return in_time, out_time

in_time, out_time = get_in_out_time()
in_time.shape, out_time.shape

((4410, 262), (4410, 262))

In [15]:
def merge_times(in_t: pd.DataFrame, out_t: pd.DataFrame):
    # out_t and in_t have the same layout:
    #  - EmployeeID
    #  - 2015-01-01_x
    #  - 2015-01-02_x
    #  - 2015-01-03_x
    #  - ...

    # We want to merge them on EmployeeID and Date labels (labels are the same)
    # We can merge each row with the corresponding row, and it would be better if the layout looked like this:
    # - EmployeeID
    # - in_2015-01-01_x
    # - out_2015-01-01_x
    # - in_2015-01-02_x
    # - out_2015-01-02_x
    # - ...

    in_t = in_t.copy()
    out_t = out_t.copy()

    # rename columns to in_ and out_ to make it easier to understand (keep EmployeeID as is)
    in_t.columns = ['EmployeeID'] + ['in_' + col for col in in_t.columns[1:]]
    out_t.columns = ['EmployeeID'] + \
        ['out_' + col for col in out_t.columns[1:]]

    in_out_times = pd.merge(in_t, out_t, on='EmployeeID')
    return in_out_times


in_out_time = merge_times(in_time, out_time)

PROCESS_DIR = 'data/processed'
if not os.path.exists(PROCESS_DIR):
    os.makedirs(PROCESS_DIR)

in_out_time.to_csv(os.path.join(PROCESS_DIR, 'in_out_time.csv'), index=False)
in_out_time.head(1)


TypeError: Index(...) must be called with a collection of some kind, False was passed